In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/gpt2-noticias/paper/Utils/* /content

In [3]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
!cd fastText-0.9.2 && pip install .

--2023-11-26 04:00:37--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/refs/tags/v0.9.2 [following]
--2023-11-26 04:00:37--  https://codeload.github.com/facebookresearch/fastText/zip/refs/tags/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip’

v0.9.2.zip              [  <=>               ]   4.17M  15.1MB/s    in 0.3s    

2023-11-26 04:00:38 (15.1 MB/s) - ‘v0.9.2.zip’ saved [4369852]

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
   creating: fastText-0.9.2/
   creating: fastText-0.9.2/.circl

In [4]:
from sklearn.model_selection import train_test_split
import WordEmbeddings as we
import LoadData as load
import pandas as pd
import FileUtil as f

In [5]:
def concatLists(l1,l2):
    l3 = []
    for i,d in enumerate(l1):
        l3.append('__label__'+str(l2[i]) + ' ' +str(l1[i]))
    return l3


In [78]:
dataSource = 'news'
random_state = 42
THRESHOLD = 50;
ver_dataset =  'pierre-final'

In [79]:
df1 = pd.DataFrame({'text': open("/content/drive/MyDrive/gpt2-noticias/dataset_pos.txt", "r").readlines()})
df1['sentiment'] = int(1)
df0 = pd.DataFrame({'text':open("/content/drive/MyDrive/gpt2-noticias/dataset_neg.txt", "r").readlines()})
df0['sentiment'] = int(0)

df_orig = pd.concat([df1,df0])
df = pd.DataFrame({'text':[],'sentiment':[]})

In [80]:
for i in range (THRESHOLD, 100, 10):
  df1_art = pd.DataFrame({'text': open(f"/content/drive/MyDrive/gpt2-noticias/dataset_p_artf{ver_dataset}_{i}.txt", "r").readlines()})
  df1_art['sentiment'] = int(1)
  df = pd.concat([df, df1_art]);

  df0_art = pd.DataFrame({'text':open(f"/content/drive/MyDrive/gpt2-noticias/dataset_n_artf{ver_dataset}_{i}.txt", "r").readlines()})
  df0_art['sentiment'] = int(0)
  df = pd.concat([df, df0_art]);

In [81]:
df['sentiment'] = df['sentiment'].astype('int')

In [82]:
data = df['text'].tolist()
labels = df['sentiment'].tolist()

data_orig = df_orig['text'].tolist()
labels_orig = df_orig['sentiment'].tolist()

In [83]:
train_x_orig, test_x_orig, train_y_orig, test_y_orig = train_test_split(data_orig,labels_orig,stratify=labels_orig,test_size=0.2, random_state = random_state)

if len(data) > 0 :
  train_x, test_x, train_y, test_y = train_test_split(data,labels,stratify=labels,test_size=0.2, random_state = random_state)
else:
  train_x, test_x, train_y, test_y = [],[],[],[]

In [84]:
train_x = train_x + train_x_orig
train_y = train_y + train_y_orig
test_x = test_x + test_x_orig
test_y = test_y + test_y_orig

In [85]:
print(f'total de amostras:{len(train_x+test_x)} (+: {df[df["sentiment"] == 1].shape[0]+df_orig[df_orig["sentiment"] == 1].shape[0]} -: {df[df["sentiment"] == 0].shape[0]+df_orig[df_orig["sentiment"] == 0].shape[0]})')

total de amostras:141359 (+: 71477 -: 69882)


In [86]:
training_path = './trainFT.txt'
test_path = './testFT.txt'

f.saveListToFile(training_path,concatLists(train_x,train_y)) #train
f.saveListToFile(test_path, concatLists(test_x,test_y)) #test

In [87]:
model = we.fastText_Classification(training_path, test_path)

In [88]:
result = model.test(training_path)
validation = model.test(test_path)

In [89]:
pred = []
for i, d in enumerate(test_x_orig):
    p = model.predict(d.strip())
    if '__label__1' in p[0] : pred.append(int(1))
    elif '__label__0' in p[0] : pred.append(int(0))
   # print(d,' - Pred:' ,p[0],' Corr: ', test_y[i])

In [90]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print('Acc: '+str(accuracy_score(test_y_orig, pred))+'\n'+
      'Precision: '+ str(precision_score(test_y_orig, pred))+'\n'+
      'Recall: '+str(recall_score(test_y_orig, pred))+'\n'+
      'F1: '+str(f1_score(test_y_orig, pred))
      )

model.save_model('./FastText_'+dataSource+'.mod')

Acc: 0.7461538461538462
Precision: 0.6857142857142857
Recall: 0.8135593220338984
F1: 0.744186046511628


In [91]:
len(pred)

130